# Hugging Face Online

Los modelos de Hugging Face están disponibles para descargar en local, y algunos incluso permiten realizar inferencias "online". 
Esto es una gran ventaja ya que nos permite crear aplicaciones analíticas sin necesidad de ocupar un gran espacio de disco.

Para ello, podemos ver un ejemplo de modelo en este [LINK](https://huggingface.co/docs/api-inference/index)

Los modelos se pueden utilizar para diferentes casos de uso (predicción, análisis de sentimiento, resumir, etc.). Algunos ejemplos los vemos a continuación


# Predicción del texto faltante

Model : https://huggingface.co/bert-base-cased?text=Paris+is+the+%5BMASK%5D+of+France. 

In [7]:
# Configuración 
import requests
import json

# Crear tu propio API key en Hugging Face es muy sencillo y Gratis!
API_KEY = "TU-API-KEY"


In [8]:
API_URL = "https://api-inference.huggingface.co/models/bert-base-cased"

headers = {"Authorization": f"Bearer {API_KEY}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": "The answer to the universe is [MASK].",
})

output


{'error': 'Model bert-base-cased is currently loading', 'estimated_time': 20.0}

In [9]:
output = query({
	"inputs": "The strongest super hero is [MASK].",
})

output

[{'score': 0.0655432865023613,
  'token': 10631,
  'token_str': 'Superman',
  'sequence': 'The strongest super hero is Superman.'},
 {'score': 0.01948622055351734,
  'token': 8622,
  'token_str': 'Batman',
  'sequence': 'The strongest super hero is Batman.'},
 {'score': 0.015481239184737206,
  'token': 19408,
  'token_str': 'Hulk',
  'sequence': 'The strongest super hero is Hulk.'},
 {'score': 0.015316893346607685,
  'token': 10670,
  'token_str': 'Zero',
  'sequence': 'The strongest super hero is Zero.'},
 {'score': 0.013313977047801018,
  'token': 16496,
  'token_str': 'Hercules',
  'sequence': 'The strongest super hero is Hercules.'}]

In [10]:
output = query({
	"inputs": "Quito is the [MASK] of Ecuador.",
})

output

[{'score': 0.9953473210334778,
  'token': 2364,
  'token_str': 'capital',
  'sequence': 'Quito is the capital of Ecuador.'},
 {'score': 0.0013217513915151358,
  'token': 6299,
  'token_str': 'Capital',
  'sequence': 'Quito is the Capital of Ecuador.'},
 {'score': 0.0003529535897541791,
  'token': 1946,
  'token_str': 'seat',
  'sequence': 'Quito is the seat of Ecuador.'},
 {'score': 0.00033725282992236316,
  'token': 2057,
  'token_str': 'center',
  'sequence': 'Quito is the center of Ecuador.'},
 {'score': 0.0002754410379566252,
  'token': 1331,
  'token_str': 'city',
  'sequence': 'Quito is the city of Ecuador.'}]

# Q&A Basado en contexto

In [11]:

API_URL = "https://api-inference.huggingface.co/models/deepset/roberta-base-squad2"
headers = {"Authorization": f"Bearer {API_KEY}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
def q_a(context: str):
  q = input()
  return query({
	"inputs": {
		"question": q,
		"context": context
	},
})['answer']

context = """
  Ecuador is the most beautiful country in the world.
  Ecuador has beaches, mountains and jungle. 
  The capital of Ecuador is Quito.
  The people in Ecuador are happy and generous.
  """

In [12]:
q_a(context)

'Ecuador'

# Sacar los Embeddings de las palabras

In [13]:

def query(payload):
    import requests
    API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/clips/mfaq"
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def get_embeding(text):
    print("Using Huggingface Inference API")
    output = query({
        "inputs": text,
        "options": {"wait_for_model": True}
    })
    return output

embedding = get_embeding("Ecuador is the most beautiful country")


Using Huggingface Inference API


In [14]:
embedding[:10]

[-0.055637046694755554,
 0.13247257471084595,
 -0.009967251680791378,
 0.29034724831581116,
 0.011160862632095814,
 -0.030794207006692886,
 -0.11202432960271835,
 0.08252018690109253,
 -0.37208491563796997,
 0.10821767151355743]

# Resumenes de textos largos 

In [15]:

# Replace 'model_name' with the name of the pre-trained summarization model you want to use
model_name = 'sshleifer/distilbart-cnn-12-6'

# Replace 'text_to_summarize' with the text you want to summarize
text_to_summarize = 'Hugging Face is a company that specializes in Natural Language Processing (NLP). Their Inference API allows developers to easily use pre-trained NLP models to perform various NLP tasks such as Named Entity Recognition (NER), Question Answering (QA), and Summarization. In this example, we will use the Hugging Face Inference API to summarize a text.'

# Define the API endpoint
endpoint = f'https://api-inference.huggingface.co/models/{model_name}'

# Set the request headers
headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

# Set the request data
data = {
    'inputs': text_to_summarize,
    'parameters': {
        'max_length': 100,
        'min_length': 20,
        'do_sample': False
    }
}

# Send the request
response = requests.post(endpoint, headers=headers, json=data)

# Get the summarized text from the response
summarized_text = response.json()[0]['summary_text']

# Print the summarized text
print(summarized_text)


 Hugging Face is a company that specializes in Natural Language Processing (NLP) Their Inference API allows developers to easily use pre-trained NLP models to perform various NLP tasks such as Named Entity Recognition (NER) and Question Answering (QA)
